## This is a starter notebook for HackLive-3 NLP on Analytics Vidhya.<br>
### If you find this notebook useful, please like and support this.
[https://datahack.analyticsvidhya.com/contest/hacklive-3-guided-hackathon-text-classification/](http://) <br>
This solution gave a rank of 27 on LB.

# Importing necessary packages and modules

 **We will be using f1_score as metrics and simpletrandformers 'RoBERTa' for model building**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from google.colab import files
!pip install simpletransformers
from simpletransformers.classification import MultiLabelClassificationModel
import logging

from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

Collecting simpletransformers
  Downloading https://files.pythonhosted.org/packages/56/35/31022262786f4aa070fe472677cea66fade8d221181a86825096af021e2c/simpletransformers-0.48.14-py3-none-any.whl (214kB)
     |████████████████████████████████| 215kB 4.7MB/s 
Collecting wandb
  Downloading https://files.pythonhosted.org/packages/af/5a/59c6f39d8aceb2c38d6090f947fd77c1cedddf44b1be728d1ae682d09d15/wandb-0.10.7-py2.py3-none-any.whl (1.7MB)
     |████████████████████████████████| 1.7MB 13.5MB/s 
Requirement already satisfied: regex in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (2019.12.20)
Collecting tensorboardx
  Downloading https://files.pythonhosted.org/packages/af/0c/4f41bcd45db376e6fe5c619c01100e9b7531c55791b7244815bac6eac32c/tensorboardX-2.1-py2.py3-none-any.whl (308kB)
     |████████████████████████████████| 317kB 30.6MB/s 
Requirement already satisfied: scikit-learn in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (0.22.2.post1)
Collecting tokenizers
  Downloading https://files.pythonhosted.org/packages/7c/a5/78be1a55b2ac8d6a956f0a211d372726e2b1dd2666bb537fea9b03abd62c/tokenizers-0.9.2-cp36-cp36m-manylinux1_x86_64.whl (2.9MB)
     |████████████████████████████████| 2.9MB 29.8MB/s 
Collecting streamlit
  Downloading https://files.pythonhosted.org/packages/c7/d2/3228e647441606a38ce4f73a30ba8a5d36a7be421123d129f7e40348dc02/streamlit-0.69.2-py2.py3-none-any.whl (7.4MB)
     |████████████████████████████████| 7.4MB 50.1MB/s 
Collecting tqdm>=4.47.0
  Downloading https://files.pythonhosted.org/packages/bd/cf/f91813073e4135c1183cadf968256764a6fe4e35c351d596d527c0540461/tqdm-4.50.2-py2.py3-none-any.whl (70kB)
     |████████████████████████████████| 71kB 10.7MB/s 
Requirement already satisfied: requests in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (2.23.0)
Requirement already satisfied: pandas in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (1.1.2)
Collecting transformers>=3.0.2
  Downloading https://files.pythonhosted.org/packages/2c/4e/4f1ede0fd7a36278844a277f8d53c21f88f37f3754abf76a5d6224f76d4a/transformers-3.4.0-py3-none-any.whl (1.3MB)
     |████████████████████████████████| 1.3MB 55.9MB/s 
Requirement already satisfied: scipy in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (1.4.1)
Requirement already satisfied: numpy in /usr/local/lib/python3.6/dist-packages (from simpletransformers) (1.18.5)
Collecting seqeval
  Downloading https://files.pythonhosted.org/packages/c4/47/f85f522f1f2532ca650474088e4024a9f52d524379bff058eaadf53cb663/seqeval-1.2.1.tar.gz (43kB)
     |████████████████████████████████| 51kB 8.3MB/s 
Requirement already satisfied: PyYAML in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (3.13)
Requirement already satisfied: protobuf>=3.12.0 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (3.12.4)
Requirement already satisfied: six>=1.13.0 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (1.15.0)
Requirement already satisfied: Click>=7.0 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (7.1.2)
Requirement already satisfied: promise<3,>=2.0 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (2.3)
Collecting subprocess32>=3.5.3
  Downloading https://files.pythonhosted.org/packages/32/c8/564be4d12629b912ea431f1a50eb8b3b9d00f1a0b1ceff17f266be190007/subprocess32-3.5.4.tar.gz (97kB)
     |████████████████████████████████| 102kB 12.4MB/s 
Collecting docker-pycreds>=0.4.0
  Downloading https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl
Collecting watchdog>=0.8.3
  Downloading https://files.pythonhosted.org/packages/0e/06/121302598a4fc01aca942d937f4a2c33430b7181137b35758913a8db10ad/watchdog-0.10.3.tar.gz (94kB)
     |████████████████████████████████| 102kB 12.4MB/s 
Collecting configparser>=3.8.1
  Downloading https://files.pythonhosted.org/packages/08/b2/ef713e0e67f6e7ec7d59aea3ee78d05b39c15930057e724cc6d362a8c3bb/configparser-5.0.1-py3-none-any.whl
Requirement already satisfied: python-dateutil>=2.6.1 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (2.8.1)
Requirement already satisfied: psutil>=5.0.0 in /usr/local/lib/python3.6/dist-packages (from wandb->simpletransformers) (5.4.8)
Collecting shortuuid>=0.5.0
  Downloading https://files.pythonhosted.org/packages/25/a6/2ecc1daa6a304e7f1b216f0896b26156b78e7c38e1211e9b798b4716c53d/shortuuid-1.0.1-py3-none-any.whl
Collecting GitPython>=1.0.0
  Downloading https://files.pythonhosted.org/packages/c0/d7/b2b0672e0331567157adf9281f41ee731c412ee518ca5e6552c27fa73c91/GitPython-3.1.9-py3-none-any.whl (159kB)
     |████████████████████████████████| 163kB 54.3MB/s 
Collecting sentry-sdk>=0.4.0
  Downloading https://files.pythonhosted.org/packages/fc/f0/188b8321988c739ab79577269c169d0d5f36aeb7172c17cd45eb0917823d/sentry_sdk-0.19.1-py2.py3-none-any.whl (120kB)
     |████████████████████████████████| 122kB 62.3MB/s 
Requirement already satisfied: joblib>=0.11 in /usr/local/lib/python3.6/dist-packages (from scikit-learn->simpletransformers) (0.16.0)
Requirement already satisfied: altair>=3.2.0 in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (4.1.0)
Collecting enum-compat
  Downloading https://files.pythonhosted.org/packages/55/ae/467bc4509246283bb59746e21a1a2f5a8aecbef56b1fa6eaca78cd438c8b/enum_compat-0.0.3-py3-none-any.whl
Collecting validators
  Downloading https://files.pythonhosted.org/packages/41/4a/3360ff3cf2b4a1b9721ac1fbff5f84663f41047d9874b3aa1ac82e862c44/validators-0.18.1-py3-none-any.whl
Requirement already satisfied: astor in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (0.8.1)
Collecting blinker
  Downloading https://files.pythonhosted.org/packages/1b/51/e2a9f3b757eb802f61dc1f2b09c8c99f6eb01cf06416c0671253536517b6/blinker-1.4.tar.gz (111kB)
     |████████████████████████████████| 112kB 48.7MB/s 
Collecting pydeck>=0.1.dev5
  Downloading https://files.pythonhosted.org/packages/51/1e/296f4108bf357e684617a776ecaf06ee93b43e30c35996dfac1aa985aa6c/pydeck-0.5.0b1-py2.py3-none-any.whl (4.4MB)
     |████████████████████████████████| 4.4MB 48.0MB/s 
Requirement already satisfied: tornado>=5.0 in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (5.1.1)
Requirement already satisfied: toml in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (0.10.1)
Collecting boto3
  Downloading https://files.pythonhosted.org/packages/30/3c/c9b65c398de6589c93b3c65aa27a695a6cdf7588c50a7934ad3ac6599a8a/boto3-1.16.2-py2.py3-none-any.whl (129kB)
     |████████████████████████████████| 133kB 57.8MB/s 
Requirement already satisfied: packaging in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (20.4)
Requirement already satisfied: pyarrow in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (0.14.1)
Requirement already satisfied: pillow>=6.2.0 in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (7.0.0)
Requirement already satisfied: cachetools>=4.0 in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (4.1.1)
Collecting botocore>=1.13.44
  Downloading https://files.pythonhosted.org/packages/1f/96/35fa364675cf17e3a190ae08716ac4b078ca86a62ef071d32cd8b86d52bc/botocore-1.19.2-py2.py3-none-any.whl (6.7MB)
     |████████████████████████████████| 6.7MB 47.4MB/s 
Requirement already satisfied: tzlocal in /usr/local/lib/python3.6/dist-packages (from streamlit->simpletransformers) (1.5.1)
Collecting base58
  Downloading https://files.pythonhosted.org/packages/3c/03/58572025c77b9e6027155b272a1b96298e711cd4f95c24967f7137ab0c4b/base58-2.0.1-py3-none-any.whl
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->simpletransformers) (2020.6.20)
Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests->simpletransformers) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests->simpletransformers) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests->simpletransformers) (1.24.3)
Requirement already satisfied: pytz>=2017.2 in /usr/local/lib/python3.6/dist-packages (from pandas->simpletransformers) (2018.9)
Collecting sentencepiece!=0.1.92
  Downloading https://files.pythonhosted.org/packages/d4/a4/d0a884c4300004a78cca907a6ff9a5e9fe4f090f5d95ab341c53d28cbc58/sentencepiece-0.1.91-cp36-cp36m-manylinux1_x86_64.whl (1.1MB)
     |████████████████████████████████| 1.1MB 51.1MB/s 
Requirement already satisfied: dataclasses; python_version < "3.7" in /usr/local/lib/python3.6/dist-packages (from transformers>=3.0.2->simpletransformers) (0.7)
Requirement already satisfied: filelock in /usr/local/lib/python3.6/dist-packages (from transformers>=3.0.2->simpletransformers) (3.0.12)
Collecting sacremoses
  Downloading https://files.pythonhosted.org/packages/7d/34/09d19aff26edcc8eb2a01bed8e98f13a1537005d31e95233fd48216eed10/sacremoses-0.0.43.tar.gz (883kB)
     |████████████████████████████████| 890kB 50.6MB/s 
Requirement already satisfied: setuptools in /usr/local/lib/python3.6/dist-packages (from protobuf>=3.12.0->wandb->simpletransformers) (50.3.0)
Collecting pathtools>=0.1.1
  Downloading https://files.pythonhosted.org/packages/e7/7f/470d6fcdf23f9f3518f6b0b76be9df16dcc8630ad409947f8be2eb0ed13a/pathtools-0.1.2.tar.gz
Collecting gitdb<5,>=4.0.1
  Downloading https://files.pythonhosted.org/packages/48/11/d1800bca0a3bae820b84b7d813ad1eff15a48a64caea9c823fc8c1b119e8/gitdb-4.0.5-py3-none-any.whl (63kB)
     |████████████████████████████████| 71kB 11.2MB/s 
Requirement already satisfied: toolz in /usr/local/lib/python3.6/dist-packages (from altair>=3.2.0->streamlit->simpletransformers) (0.11.1)
Requirement already satisfied: jsonschema in /usr/local/lib/python3.6/dist-packages (from altair>=3.2.0->streamlit->simpletransformers) (2.6.0)
Requirement already satisfied: entrypoints in /usr/local/lib/python3.6/dist-packages (from altair>=3.2.0->streamlit->simpletransformers) (0.3)
Requirement already satisfied: jinja2 in /usr/local/lib/python3.6/dist-packages (from altair>=3.2.0->streamlit->simpletransformers) (2.11.2)
Requirement already satisfied: decorator>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from validators->streamlit->simpletransformers) (4.4.2)
Requirement already satisfied: traitlets>=4.3.2 in /usr/local/lib/python3.6/dist-packages (from pydeck>=0.1.dev5->streamlit->simpletransformers) (4.3.3)
Requirement already satisfied: ipywidgets>=7.0.0 in /usr/local/lib/python3.6/dist-packages (from pydeck>=0.1.dev5->streamlit->simpletransformers) (7.5.1)
Collecting ipykernel>=5.1.2; python_version >= "3.4"
  Downloading https://files.pythonhosted.org/packages/52/19/c2812690d8b340987eecd2cbc18549b1d130b94c5d97fcbe49f5f8710edf/ipykernel-5.3.4-py3-none-any.whl (120kB)
     |████████████████████████████████| 122kB 60.1MB/s 
Collecting jmespath<1.0.0,>=0.7.1
  Downloading https://files.pythonhosted.org/packages/07/cb/5f001272b6faeb23c1c9e0acc04d48eaaf5c862c17709d20e3469c6e0139/jmespath-0.10.0-py2.py3-none-any.whl
Collecting s3transfer<0.4.0,>=0.3.0
  Downloading https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl (69kB)
     |████████████████████████████████| 71kB 10.7MB/s 
Requirement already satisfied: pyparsing>=2.0.2 in /usr/local/lib/python3.6/dist-packages (from packaging->streamlit->simpletransformers) (2.4.7)
Collecting smmap<4,>=3.0.1
  Downloading https://files.pythonhosted.org/packages/b0/9a/4d409a6234eb940e6a78dfdfc66156e7522262f5f2fecca07dc55915952d/smmap-3.0.4-py2.py3-none-any.whl
Requirement already satisfied: MarkupSafe>=0.23 in /usr/local/lib/python3.6/dist-packages (from jinja2->altair>=3.2.0->streamlit->simpletransformers) (1.1.1)
Requirement already satisfied: ipython-genutils in /usr/local/lib/python3.6/dist-packages (from traitlets>=4.3.2->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.2.0)
Requirement already satisfied: ipython>=4.0.0; python_version >= "3.3" in /usr/local/lib/python3.6/dist-packages (from ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (5.5.0)
Requirement already satisfied: widgetsnbextension~=3.5.0 in /usr/local/lib/python3.6/dist-packages (from ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (3.5.1)
Requirement already satisfied: nbformat>=4.2.0 in /usr/local/lib/python3.6/dist-packages (from ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (5.0.7)
Requirement already satisfied: jupyter-client in /usr/local/lib/python3.6/dist-packages (from ipykernel>=5.1.2; python_version >= "3.4"->pydeck>=0.1.dev5->streamlit->simpletransformers) (5.3.5)
Requirement already satisfied: pexpect; sys_platform != "win32" in /usr/local/lib/python3.6/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (4.8.0)
Requirement already satisfied: simplegeneric>0.8 in /usr/local/lib/python3.6/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.8.1)
Requirement already satisfied: prompt-toolkit<2.0.0,>=1.0.4 in /usr/local/lib/python3.6/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (1.0.18)
Requirement already satisfied: pickleshare in /usr/local/lib/python3.6/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.7.5)
Requirement already satisfied: pygments in /usr/local/lib/python3.6/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (2.6.1)
Requirement already satisfied: notebook>=4.4.1 in /usr/local/lib/python3.6/dist-packages (from widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (5.3.1)
Requirement already satisfied: jupyter-core in /usr/local/lib/python3.6/dist-packages (from nbformat>=4.2.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (4.6.3)
Requirement already satisfied: pyzmq>=13 in /usr/local/lib/python3.6/dist-packages (from jupyter-client->ipykernel>=5.1.2; python_version >= "3.4"->pydeck>=0.1.dev5->streamlit->simpletransformers) (19.0.2)
Requirement already satisfied: ptyprocess>=0.5 in /usr/local/lib/python3.6/dist-packages (from pexpect; sys_platform != "win32"->ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.6.0)
Requirement already satisfied: wcwidth in /usr/local/lib/python3.6/dist-packages (from prompt-toolkit<2.0.0,>=1.0.4->ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.2.5)
Requirement already satisfied: terminado>=0.8.1 in /usr/local/lib/python3.6/dist-packages (from notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.9.1)
Requirement already satisfied: nbconvert in /usr/local/lib/python3.6/dist-packages (from notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (5.6.1)
Requirement already satisfied: Send2Trash in /usr/local/lib/python3.6/dist-packages (from notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (1.5.0)
Requirement already satisfied: testpath in /usr/local/lib/python3.6/dist-packages (from nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.4.4)
Requirement already satisfied: defusedxml in /usr/local/lib/python3.6/dist-packages (from nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.6.0)
Requirement already satisfied: mistune<2,>=0.8.1 in /usr/local/lib/python3.6/dist-packages (from nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.8.4)
Requirement already satisfied: bleach in /usr/local/lib/python3.6/dist-packages (from nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (3.2.1)
Requirement already satisfied: pandocfilters>=1.4.1 in /usr/local/lib/python3.6/dist-packages (from nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (1.4.2)
Requirement already satisfied: webencodings in /usr/local/lib/python3.6/dist-packages (from bleach->nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->pydeck>=0.1.dev5->streamlit->simpletransformers) (0.5.1)
Building wheels for collected packages: seqeval, subprocess32, watchdog, blinker, sacremoses, pathtools
  Building wheel for seqeval (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.1-cp36-none-any.whl size=16168 sha256=9ed1ab4c6131504db6d5380f3549630f9cc9e89988bb6a37acc6b821dabb7ebe
  Stored in directory: /root/.cache/pip/wheels/b4/42/c0/e2d946497f714404c0b78a0ab4c7f45222a293f49320932463
  Building wheel for subprocess32 (setup.py) ... done
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=89ee857ef6690717eee83e16af91991e657cd1cec243181c2c5d49966d8be3be
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Building wheel for watchdog (setup.py) ... done
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=ebef76ce9841d95ccd920ad53386dde2ea045ea8d1c6916d43629b24d40f61a5
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Building wheel for blinker (setup.py) ... done
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=63035143f4ccf25e01d9e05d364c990ddffd9e1f58f3da985e28267fddb9e301
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Building wheel for sacremoses (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cff37755b3eba0ac092609992b99486e3611e8752b903b9f0123898972e5c8c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Building wheel for pathtools (setup.py) ... done
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=8052a65e19fedce779e42bd5e4c74d0651e9a7d86f087a49842560cb1a05d096
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built seqeval subprocess32 watchdog blinker sacremoses pathtools
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
ERROR: botocore 1.19.2 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
ERROR: seqeval 1.2.1 has requirement numpy==1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: seqeval 1.2.1 has requirement scikit-learn==0.23.2, but you'll have scikit-learn 0.22.2.post1 which is incompatible.
Installing collected packages: subprocess32, docker-pycreds, pathtools, watchdog, configparser, shortuuid, smmap, gitdb, GitPython, sentry-sdk, wandb, tensorboardx, tokenizers, enum-compat, validators, blinker, ipykernel, pydeck, jmespath, botocore, s3transfer, boto3, base58, streamlit, tqdm, sentencepiece, sacremoses, transformers, seqeval, simpletransformers
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
Successfully installed GitPython-3.1.9 base58-2.0.1 blinker-1.4 boto3-1.16.2 botocore-1.19.2 configparser-5.0.1 docker-pycreds-0.4.0 enum-compat-0.0.3 gitdb-4.0.5 ipykernel-5.3.4 jmespath-0.10.0 pathtools-0.1.2 pydeck-0.5.0b1 s3transfer-0.3.3 sacremoses-0.0.43 sentencepiece-0.1.91 sentry-sdk-0.19.1 seqeval-1.2.1 shortuuid-1.0.1 simpletransformers-0.48.14 smmap-3.0.4 streamlit-0.69.2 subprocess32-3.5.4 tensorboardx-2.1 tokenizers-0.9.2 tqdm-4.50.2 transformers-3.4.0 validators-0.18.1 wandb-0.10.7 watchdog-0.10.3
WARNING: Upgrading ipython, ipykernel, tornado, prompt-toolkit or pyzmq can
cause your runtime to repeatedly crash or behave in unexpected ways and is not
recommended. If your runtime won't connect or execute code, you can reset it
with "Factory reset runtime" from the "Runtime" menu.
WARNING: The following packages were previously imported in this runtime:
  [ipykernel]
You must restart the runtime in order to use newly installed versions.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.

Getting the train, test and submission data

In [ ]:
train = pd.read_csv('../input/hacklive-3-nlp/Train.csv')
test = pd.read_csv('../input/hacklive-3-nlp/Test.csv')
ss = pd.read_csv('../input/hacklive-3-nlp/SampleSubmission_Uqu2HVA.csv')

Cleaning the data

In [ ]:
import re

train.loc[:,"ABSTRACT"] = train.ABSTRACT.apply(lambda x : " ".join(re.findall('[\w]+',x)))
test.loc[:,"ABSTRACT"] = test.ABSTRACT.apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [ ]:
ID_COL = 'id'

TARGET_COLS = ['Analysis of PDEs', 'Applications',
               'Artificial Intelligence', 'Astrophysics of Galaxies',
               'Computation and Language', 'Computer Vision and Pattern Recognition',
               'Cosmology and Nongalactic Astrophysics',
               'Data Structures and Algorithms', 'Differential Geometry',
               'Earth and Planetary Astrophysics', 'Fluid Dynamics',
               'Information Theory', 'Instrumentation and Methods for Astrophysics',
               'Machine Learning', 'Materials Science', 'Methodology', 'Number Theory',
               'Optimization and Control', 'Representation Theory', 'Robotics',
               'Social and Information Networks', 'Statistics Theory',
               'Strongly Correlated Electrons', 'Superconductivity',
               'Systems and Control']

TOPIC_COLS = ['Computer Science', 'Mathematics', 'Physics', 'Statistics']

# **Threshold Optimization**

In [ ]:
def get_best_thresholds(true, preds):
    
    thresholds = [i/100 for i in range(100)]
    best_thresholds = []
    for idx in range(25):
        
        f1_scores = [f1_score(true[:, idx], (preds[:, idx] > thresh) * 1) for thresh in thresholds]
        best_thresh = thresholds[np.argmax(f1_scores)]
        best_thresholds.append(best_thresh)
    return best_thresholds

Joining the Topic columns to ABSTRACT to use in MultiLabelClassificationModel

In [ ]:
l1 = train[TARGET_COLS].values.tolist()

for col in TOPIC_COLS:
    train[col] = train[col].map({1:col, 0:'.'})
    test[col] = test[col].map({1:col, 0:'.'})
    
for col in TOPIC_COLS:
    train['ABSTRACT'] = train['ABSTRACT'] +'.'+ train[col]
    test['ABSTRACT'] = test['ABSTRACT'] +'.'+ test[col]

**Splitting the data into train and eval dataframes**

In [ ]:
train['labels'] = l1
train.rename(columns={'ABSTRACT':'text'}, inplace=True)
train_data = train[['text','labels']]
train_df = train_data.iloc[:12000, :]
eval_df = train_data.iloc[12000:, :]

In [ ]:
train_df.head()

In [ ]:
test_df = test['ABSTRACT']

# Model Building

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=25, use_cuda=True, args={'reprocess_input_data':True,
                                                                                      'overwrite_output_dir':True,
                                                                                      'num_train_epochs':8,
                                                                                      'early_stopping_consider_epochs':True,
                                                                                      'early_stopping_patience':True})

model.train_model(train_df, acc=f1_score)
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)

INFO:filelock:Lock 140682372578944 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
Downloading: 100%
481/481 [00:00<00:00, 2.58kB/s]
INFO:filelock:Lock 140682372578944 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:filelock:Lock 140683356743776 acquired on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock

Downloading: 100%
501M/501M [00:10<00:00, 49.8MB/s]
INFO:filelock:Lock 140683356743776 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:filelock:Lock 140682335128936 acquired on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
Downloading: 100%
899k/899k [00:00<00:00, 5.34MB/s]
INFO:filelock:Lock 140682335128936 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:filelock:Lock 140682365834408 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock

Downloading: 100%
456k/456k [00:00<00:00, 3.68MB/s]
INFO:filelock:Lock 140682365834408 released on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%
12000/12000 [03:26<00:00, 57.98it/s]

Epoch 8 of 8: 100%
8/8 [27:26<00:00, 205.82s/it]
Epochs 0/8. Running Loss: 0.1045: 100%
1500/1500 [08:28<00:00, 2.95it/s]
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
Epochs 1/8. Running Loss: 0.0660: 100%
1500/1500 [05:24<00:00, 4.62it/s]

Epochs 2/8. Running Loss: 0.0655: 100%
1500/1500 [06:30<00:00, 3.84it/s]

Epochs 3/8. Running Loss: 0.0487: 100%
1500/1500 [06:52<00:00, 3.64it/s]

Epochs 4/8. Running Loss: 0.0304: 100%
1500/1500 [06:25<00:00, 3.89it/s]

Epochs 5/8. Running Loss: 0.0635: 100%
1500/1500 [09:40<00:00, 2.58it/s]

Epochs 6/8. Running Loss: 0.0205: 100%
1500/1500 [06:25<00:00, 3.89it/s]

Epochs 7/8. Running Loss: 0.0096: 100%
1500/1500 [04:00<00:00, 6.24it/s]


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
100%
2004/2004 [06:49<00:00, 4.90it/s]

Running Evaluation: 100%
251/251 [00:44<00:00, 5.60it/s]

{'LRAP': 0.853300918282204, 'eval_loss': 0.08591521177737718}
[[1.23023987e-03 7.54928589e-03 2.86674500e-03 ... 3.73649597e-03
  3.33786011e-03 2.88963318e-03]
 [3.37219238e-02 5.24139404e-03 3.18527222e-03 ... 3.02734375e-02
  1.54266357e-02 4.90570068e-03]
 [3.14950943e-04 1.42669678e-03 2.23510742e-01 ... 8.33034515e-04
  3.43322754e-04 1.00040436e-03]
 ...
 [2.81143188e-03 1.59168243e-03 3.02696228e-03 ... 3.40087891e-01
  9.74609375e-01 2.10762024e-03]
 [3.16047668e-03 1.56707764e-02 4.89257812e-01 ... 6.79779053e-03
  1.27887726e-03 1.64890289e-03]
 [2.42424011e-03 1.44958496e-03 3.88526917e-03 ... 9.70703125e-01
  7.51953125e-02 1.78909302e-03]]

Predicting for test data for submission

In [ ]:
predictions, raw_outputs = model.predict(test_df)
#print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%
6002/6002 [04:48<00:00, 20.77it/s]

100%
751/751 [00:21<00:00, 35.62it/s]

[[3.35454941e-04 1.37939453e-02 1.37405396e-02 ... 2.08282471e-03
  1.01184845e-03 1.46675110e-03]
 [7.94887543e-04 8.14453125e-01 2.72460938e-01 ... 5.13839722e-03
  1.70707703e-03 1.70440674e-02]
 [8.64410400e-03 6.14547729e-03 4.29916382e-03 ... 4.28009033e-03
  3.37600708e-03 4.23049927e-03]
 ...
 [8.65936279e-04 1.97982788e-03 9.20410156e-02 ... 8.86440277e-04
  1.08146667e-03 1.40476227e-03]
 [1.20162964e-03 8.03222656e-01 4.62722778e-03 ... 6.98471069e-03
  7.54928589e-03 2.10113525e-02]
 [2.16484070e-03 1.11198425e-03 1.52492523e-03 ... 6.35253906e-01
  9.58862305e-02 6.96182251e-04]]

In [ ]:
ss[TARGET_COLS] = raw_outputs

In [ ]:
ss.to_csv('transmore_epochs.csv', index = False)

Below is the .csv file that was submitted

In [ ]:
sub = pd.read_csv('../input/hacklive-3-nlp/transmore_epochs.csv')

## If you find this notebook useful, please like and support for more. 